In [0]:
import numpy as np

# Singular Value Decomposition

Let's work through Example 4.13 of [Mathematics for Machine Learning](https://mml-book.github.io/) using NumPy.

We would like to compute the singular value decomposition (SVD) of $\mathbf{A} =
\begin{bmatrix}
1  & 0 & 1\\
-2 & 1 & 0\\
\end{bmatrix}
$.

Step 1 is to compute the right-singular vectors as the eigenbasis of $\mathbf{A}^{\top}\mathbf{A}$:

In [0]:
A = np.array([[1., 0., 1.], [-2., 1., 0.]])
d, V = np.linalg.eig(np.dot(A.T, A))
print(d)
print(V)

Note that `numpy.linalg.eig` does not necessarily order its eigenvalues. By convention, we will sort the eigenvectors from largest to smallest eigenvalue:

In [0]:
idx = d.argsort()[::-1]
d = d[idx]
V = V[:, idx]
print(d)
print(V)

Step 2 is to compute the singular-value matrix. The singular values are the square roots of the eigenvalues of $\mathbf{A}^{\top}\mathbf{A}$. Since the rank of $\mathbf{A}=2$, there are only two non-zero singular values. Remember that the singular value matrix must be the same size as $\mathbf{A}$, so we pad with zeros accordingly:

In [0]:
min_dim = min(A.shape)
nz_idx = ~np.isclose(d, np.zeros_like(d))  # index to non-zero elements
Sigma = np.zeros_like(A)
# This next step is careful to construct Sigma with the right zero padding
Sigma[:min_dim, :min_dim] = np.diag(np.sqrt(d[nz_idx]))
print(Sigma)

We see that indeed $\mathbf{\Sigma}=
\begin{bmatrix}
\sqrt{6} & 0 & 0\\
0        & 1 & 0
\end{bmatrix}$.

Step 3 is to compute the left-singular vectors as the normalized image of the right-singular vectors:

In [0]:
m = A.shape[0]
U = np.zeros((m, m))
for i in range(m):
  U[:, i] = (1/Sigma.flat[::Sigma.shape[1]+1][i]) * np.dot(A, V[:, i])
print(U)

We see that indeed $\mathbf{U} =
\frac{1}{\sqrt{5}}
\begin{bmatrix}
1  & 2\\
-2 & 1
\end{bmatrix}.$

Note that this procedure is numerically unstable, and we can typically compute the SVD without resorting to the eigenvalue decomposition of $\mathbf{A}^{\top}\mathbf{A}$. `numpy.linalg.svd` provides a convenient way to do this:

In [0]:
u, s, vh = np.linalg.svd(A)  # using the variable names in numpy docs
print(u)
print(s)
# The "h" in vh refers to the Hermetian, which is the complex generalization
# of the transpose
# Effectively, this is V.T
print(vh)  

You will notice, that for the exception of negative signs that appear in $\mathbf{U}$ and $\mathbf{V}^{\top}$ and cancel out, that this is the same decomposition that we computed above. We can verify that it implements the linear mapping $\mathbf{A}$:

In [0]:
smat = np.zeros_like(A)
smat[:min_dim, :min_dim] = np.diag(s)
np.allclose(u @ smat @ vh, A)  # note that @ is matrix multiplication

## Exercise

Verify that the SVD implements $\mathbf{A}$ like we did above, but without having to explicitly compute `smat`. In other words, leave `s` in vector form. Hint: what part of `vh` is not used at all in `u @ smat @ vh`?